In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import re
import time
import numpy as np
import pandas as pd

In [ ]:
import tensorflow as tf

In [ ]:
df2 = pd.read_excel('data_final_consolidada_movies.xlsx')
df=df2[df2['Nombre_closter'].isin(["Cluster_3","Cluster_4"])]
df.head()

,name,promedio_ganancia_x,calificacion1_dec_x,calificacion2_dec_x,cluster,pp,Nombre_closter,budget,company,country,director,genre,gross,rating,day_released,month_released,year_released,released,runtime,score,star,votes,writer,year,hashtag,name_roten,calificacion1,calificacion2,synopsis,ruta,promedio_ganancia_y,calificacion1_dec_y,calificacion2_dec_y,max_sentimiento,min_sentimiento,mean_sentimiento,sentimiento,prueba
0,The Wolf of Wall Street,1,4.0,4.0,2,"['jordan', 'belfort', 'leonardo', 'dicaprio', ...",Cluster_3,100000000,Red Granite Pictures,USA,Martin Scorsese,Biography,116900694,R,25,12,2013,2013-12-25,180,8.2,Leonardo DiCaprio,895552,Terence Winter,2013,#TheWolfofWallStreet,the_wolf_of_wall_street,80.0,83.0,in 1987 jordan belfort leonardo dicaprio ta...,https://www.rottentomatoes.com/m/the_wolf_of_w...,1,4.0,4.0,0.8271,-0.8458,0.017757,Neutro,"['wolf', 'wall', 'street']"
1,The Conjuring,0,4.0,4.0,2,"['paranormal', 'investigators', 'demonologists...",Cluster_3,20000000,New Line Cinema,USA,James Wan,Horror,137400141,R,19,7,2013,2013-07-19,112,7.5,Patrick Wilson,342870,Chad Hayes,2013,#TheConjuring,the_conjuring,86.0,83.0,in 1970 paranormal investigators and demonolo...,https://www.rottentomatoes.com/m/the_conjuring,0,4.0,4.0,0.8658,-0.8664,0.025267,Neutro,['conjuring']
2,Prisoners,1,4.0,4.0,2,"['keller', 'dover', 'hugh', 'jackman', 'faces'...",Cluster_3,46000000,Alcon Entertainment,USA,Denis Villeneuve,Crime,61002302,R,20,9,2013,2013-09-20,153,8.1,Hugh Jackman,449489,Aaron Guzikowski,2013,#Prisoners,prisoners,81.0,87.0,keller dover hugh jackman faces a parent s w...,https://www.rottentomatoes.com/m/prisoners_2013,1,4.0,4.0,0.9405,-0.8494,-0.074207,Neutro,['prisoners']
4,Now You See Me,1,3.0,4.0,2,"['charismatic', 'magician', 'atlas', 'jesse', ...",Cluster_3,75000000,Summit Entertainment,USA,Louis Leterrier,Crime,117723989,PG-13,31,5,2013,2013-05-31,115,7.3,Jesse Eisenberg,505432,Ed Solomon,2013,#NowYouSeeMe,now_you_see_me,51.0,70.0,charismatic magician atlas jesse eisenberg l...,https://www.rottentomatoes.com/m/now_you_see_me,1,3.0,4.0,0.9455,-0.7345,0.359890,Neutro,['see']
7,We're the Millers,0,2.0,4.0,2,"['small', 'time', 'pot', 'dealer', 'david', 'j...",Cluster_3,37000000,New Line Cinema,USA,Rawson Marshall Thurber,Comedy,150394119,R,7,8,2013,2013-08-7,110,7.0,Jason Sudeikis,342500,Bob Fisher,2013,#We'retheMillers,we're_the_millers,48.0,72.0,small time pot dealer david jason sudeikis l...,https://www.rottentomatoes.com/m/were_the_millers,0,2.0,4.0,NaN,NaN,NaN,Negativo,"[""we're"", 'millers']"


In [ ]:
text = df.synopsis.str.cat(sep=" ")

In [ ]:
vocab = sorted(set(text))
print(f'{len(vocab):,.0f} caracteres únicos')

40 caracteres únicos


In [ ]:
text

'in 1987  jordan belfort  leonardo dicaprio  takes an entry level job at a wall street brokerage firm  by the early 1990s  while still in his 20s  belfort founds his own firm  stratton oakmont  together with his trusted lieutenant  jonah hill  and a merry band of brokers  belfort makes a huge fortune by defrauding wealthy investors out of millions  however  while belfort and his cronies partake in a hedonistic brew of sex  drugs and thrills  the sec and the fbi close in on his empire of excess in 1970  paranormal investigators and demonologists lorraine  vera farmiga  and ed  patrick wilson  warren are summoned to the home of carolyn  lili taylor  and roger  ron livingston  perron  the perrons and their five daughters have recently moved into a secluded farmhouse  where a supernatural presence has made itself known  though the manifestations are relatively benign at first  events soon escalate in horrifying fashion  especially after the warrens discover the house s macabre history kell

In [ ]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 64

# Number of RNN units
rnn_units = 512

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

In [ ]:
text_as_int = np.array([char2idx[c] for c in text])

In [ ]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1) # división entera

In [ ]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
    return model

In [ ]:
BATCH_SIZE = 256

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((256, 100), (256, 100)), types: (tf.int64, tf.int64)>

In [ ]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (256, None, 64)           2560      
_________________________________________________________________
gru_6 (GRU)                  (256, None, 512)          887808    
_________________________________________________________________
gru_7 (GRU)                  (256, None, 512)          1575936   
_________________________________________________________________
dense_3 (Dense)              (256, None, 40)           20520     
Total params: 2,486,824
Trainable params: 2,486,824
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(256, 100, 40) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (256, None, 64)           2560      
_________________________________________________________________
gru_6 (GRU)                  (256, None, 512)          887808    
_________________________________________________________________
gru_7 (GRU)                  (256, None, 512)          1575936   
_________________________________________________________________
dense_3 (Dense)              (256, None, 40)           20520     
Total params: 2,486,824
Trainable params: 2,486,824
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [ ]:
sampled_indices

array([19, 34, 20, 21, 36,  8, 23,  3, 23,  3, 19, 33, 26, 36, 14, 28, 21,
       33,  5,  7, 19, 32, 39, 15, 24, 12,  1, 36, 28, 24, 25,  6,  3, 17,
       18, 33, 15, 33,  9,  7, 23, 33, 34, 26, 25, 27, 39, 37, 12, 13, 30,
       12, 21, 17, 30, 39, 35, 30,  9, 26, 34, 36, 24, 14, 24, 36, 23, 32,
        9, 13, 11, 38, 14, 35, 19, 12, 15, 27, 26, 34, 10, 29, 23, 29, 20,
        1, 38, 36,  1, 28, 16, 26, 17, 33, 22,  9,  8, 13, 18, 25])

In [ ]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 's  spy satellite system  together  the former classmates encounter shootouts  espionage and double c'

Next Char Predictions: 
 'fughw7j2j2ftmwaoht46fszbkS0wokl52detbt86jtumlnzxS`qShdqzvq8muwkakwjs8`DyavfSbnmu9pjpg0yw0ocmdti87`el'


In [ ]:
# Función de perdida para optimizar el modelo
def loss(labels, logits):
      return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [ ]:
model.compile(optimizer='adam', loss=loss)


In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = 'training_checkpoints/'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_prefix,
        save_weights_only=True)

In [ ]:
# Una epoca es la cantidad de pasos completos en el conjunto de datos de entrenamiento
EPOCHS=100
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir)) # Comentar esta linea la primera vez 
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
2/2 [==============================] - 8s 4s/step - loss: 0.2140
Epoch 2/100
2/2 [==============================] - 8s 4s/step - loss: 0.2163
Epoch 3/100
2/2 [==============================] - 8s 4s/step - loss: 0.2132
Epoch 4/100
2/2 [==============================] - 8s 4s/step - loss: 0.2138
Epoch 5/100
2/2 [==============================] - 8s 4s/step - loss: 0.2120
Epoch 6/100
2/2 [==============================] - 8s 4s/step - loss: 0.2117
Epoch 7/100
2/2 [==============================] - 8s 4s/step - loss: 0.2103
Epoch 8/100
2/2 [==============================] - 9s 4s/step - loss: 0.2056
Epoch 9/100
2/2 [==============================] - 8s 4s/step - loss: 0.2089
Epoch 10/100
2/2 [==============================] - 8s 4s/step - loss: 0.2106
Epoch 11/100
2/2 [==============================] - 8s 4s/step - loss: 0.2096
Epoch 12/100
2/2 [==============================] - 9s 4s/step - loss: 0.2084
Epoch 13/100
2/2 [==============================] - 8s 4s/step - loss: 0.

In [ ]:
p_model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
p_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
p_model.build(tf.TensorShape([1, None]))

In [ ]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return(start_string + ''.join(text_generated))

In [ ]:
print(generate_text(p_model, start_string=u"fantasy"))

fantasy with a treasury agent  j k  simmoned to the help of smiritide masterpite e chen fancy    are anger to take a hart the thistr  he surkers than one of the greatest wonders of the world s friends must also cope with their own challenges  eric  kevin james  must face his ultimate fear  known as the four horsemen find themselves in more trouble in macau  china  devious tech wizard war work together to stone everyone survives and returns homel wright   boster  the two adventurers travel to the metropolis to uncover its misint war chis best friend snoopy embarks on an epic adventure in a fatthe mis who contimain in at humal caile  an durpergly  the death of the best selling  goosebumps  series  when zach accidentally raused the wak  the rebellious and mischievous boy discovers his true destiny  becoming the hero forever  ncipions a rusised to a smail tolngen in a siless  leo the assignmbak  himself vaces whan it s be a d first each other ly stown s  prosoppitict camby finding openatio

In [ ]:
print(generate_text(p_model, start_string=u"horror"))

horrory  tasus aftered the witch s beauty  a milk white cow  hair as yellow as corn  darine jount on the couple s fifth wedding anniversary  nick he s on ariss his friends embark on a journey to the sea of monsters to sandy patters into a just  nico confrontation  for the loeton for his he destroying e iiinistible allies and dangeroriconnass  sylvester stallone  co founded the expendables with conrad stonebants  to  a7s take of being an adtain for his cavers a take for the fate of humanity in flint s handed only hope with a hatikn s micad has goco above all else  hotever  e criminals for the one to bring everyone swonve his new partner  nich enger with an unusual dream  to nejaves an only supuries  arly when their wild night leads to atlanta s most notwort from a seemingly invincible enemy bullied as a teenn becomes  was to ims he takes a pad takes themamy and strengan  knows best peespecially if youly days alound the space race and galvanized the wif organization  withol king xerxes o

In [ ]:
print(generate_text(p_model, start_string=u"hero"))

herous  old school dad  mel brooks  paysage a bit the same ace brown len doves an incredible 3970  and ashy red haired girl moves into his neighborhood  charlie brown dedeave like and sursenderavel the mystery of nightshade and help them save themselves   and the world a drug add president benjamin asher  aaron eckhart  and his slate to creathergs the demendes   my setither  the e    serve as the brains behind one of the greatest operations in his poop teams when he accidentally s must jains hope to end wit  still then ice  freya teaches her youngs and bla  the tanker discovers the u s  continute rocks hannan s gomont of helovercome an urms  upbut witch has two weeks left before his mission ends and he joins hos to prestudation their commander in chief from a horrible fate young kujon fighters to teerity complem  and their three children are comfortably settled in the long on a journey to rescue her friends  their missing intermated final is crathingance their crewon for king leopold  

In [ ]:
print(generate_text(p_model, start_string=u"super"))

super  tares as a shock when he learns that jack is actually working undercover to protect komaroven and back  an unexpected turn of events creates an altered timeline  instead of dollars  the bod souns down butch cavendishaps  as this mismatched duo corming fard war  a mysterious criminal organization plans to use nuclear weapons and technology to upsethe condlown as peternifying fasiin  however  d likes with atags  connor coven as bohe to raine  a lauge and private  chinary plankton  mr  lawrence  an asher allies in april o new york langlen  after stonebanks  louis  althoond hin 20 sainsfrick  new m st peth police accounty of nmakes force atts faced with decisions that begin to frobe the warrens discover the happy  and his comrades mesmerize during a rost other  spurtakin the persian armada new york lawyer carly whitten  cameron divided by the pirate partakities  a sed findly humal cbiel three childre  richael brital busites his team decide not to kill them  jounnty warrick and growi

In [ ]:
print(generate_text(p_model, start_string=u"magic"))

magic bravery  jeff galfopfiem crash  day be wiped the heart of contains as a habriant minds during world war ii  the naziss and criminals from the streets of blu since jales  it s up to banning to save his commander in chief from a horrible fate young kubo s  art particular  and the oter  however  jason since  tries to convince his cohorts that he knows a way out tris prior  shailene woodley  lives in a sublen workingsuluntend  but withtork  jonner  war air brings him to defeat the turks   but cost him his humanity as a comes as a shocthe better  hosed obly to abt and gurblint word uit rocky balbo  sately gods on maumial rusns work together to stop the evildoers in their trackst seemingly impossible odds mr  meacham  robert redford  a woodcarver  delighting fiand and ching workens quink to gove myates a pateon for his handers a dawicaly midask an johnner  with gime aced minds during world war ii  the nazis steal countless pieces of a tay to fent the unlikeliest of heroes   and tell th

In [ ]:
print(generate_text(p_model, start_string=u"travel")) 

travel the discovery about their supposedly worked for a spy organization  lives in a south london housing estate and seems headed for a life brilliant mires  tarthy in love  as kmom to san rest of the tourists from an all out  prehistoric assault Sin Datos when tony stanking repairman  and the rest of the jous conture awaits as he meets new friend james hook  into the authur father  mark wahlies john glaber  with his beloved wife  mirena  sanahory progors about of the galactic empire  the galaxy fates a quester  more her  take  emalown as he gets tow mukine engrand in hisdon disgory persian arcadeline johnshool   is life around and channel ruins  melistanes life is good for all the food iiver with and greatest search for inner peace but legandha bount along with his son and museum friends  must travel the mystery of nightshade and her clipo to take down donovan as he battles an oppontend burswitate  and friendship  together  engroney a mexia  seiz tomntawimately arrivis best friend sn